# Complete Agentic AI Educational Assistant Implementation
# Designed for Google Colab with modular architecture


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

ValueError: mount failed

In [ ]:
import os
import json
import pandas as pd
import numpy as np
from typing import Dict, List, Any, Optional, Tuple
from dataclasses import dataclass, asdict
from datetime import datetime
import re
import pickle
from pathlib import Path

In [ ]:
!pip install --upgrade langchain langchain-community langchain-core

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.3/438.3 kB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 6.1 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.59
    Uninstalling langchain-core-0.3.59:
      Successfully uninstalled langchain-core-0.3.59


In [ ]:
# Install required packages
!pip install transformers torch sentence-transformers openai langchain chromadb pypdf2 python-docx markdown beautifulsoup4 requests

import torch
from transformers import pipeline, AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer
import chromadb
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
import PyPDF2
import docx
import markdown
from bs4 import BeautifulSoup
import requests

In [ ]:
# Configure environment
os.environ['TOKENIZERS_PARALLELISM'] = 'false'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🚀 System initialized on: {device}")

🚀 System initialized on: cpu


# 📚 DATA MODELS AND CONFIGURATION



In [ ]:
@dataclass
class ProfessorProfile:
    """Comprehensive professor pedagogy model"""
    name: str = ""
    teaching_style: str = ""  # practical/theoretical/balanced
    experience_level: str = ""  # new/experienced/veteran
    assessment_style: str = ""  # frequent_quizzes/major_exams/project_based
    emphasis_areas: List[str] = None
    teaching_methodology: str = ""
    evaluation_criteria: Dict[str, float] = None
    special_instructions: List[str] = None
    course_difficulty: str = ""  # easy/moderate/challenging
    interaction_style: str = ""  # formal/casual/interactive

    def __post_init__(self):
        if self.emphasis_areas is None:
            self.emphasis_areas = []
        if self.evaluation_criteria is None:
            self.evaluation_criteria = {}
        if self.special_instructions is None:
            self.special_instructions = []

@dataclass
class CourseContent:
    """Structure for course materials"""
    title: str
    content: str
    content_type: str  # slides/notes/policy/reference/exam
    topics: List[str] = None
    difficulty_level: str = ""

    def __post_init__(self):
        if self.topics is None:
            self.topics = []

@dataclass
class GeneratedQuestion:
    """Structure for generated questions"""
    question: str
    question_type: str  # mcq/short_answer/essay/problem_solving
    difficulty: str  # easy/medium/hard
    topic: str
    answer: str = ""
    options: List[str] = None
    explanation: str = ""
    estimated_time: int = 5  # minutes

    def __post_init__(self):
        if self.options is None:
            self.options = []


# 🧠 CORE AGENTIC AI ENGINE

In [ ]:

class AgenticEducationalAssistant:
    """
    Main agentic AI system for educational assistance
    Combines document processing, pedagogy modeling, and question generation
    """

    def __init__(self, use_perplexity_api: bool = False, api_key: str = ""):
        """Initialize the agentic AI system"""
        print("🔧 Initializing Agentic Educational Assistant...")

        # Core AI models
        self.embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
        self.text_generator = pipeline('text-generation',
                                     model='microsoft/DialoGPT-medium',
                                     device=0 if torch.cuda.is_available() else -1)

        # Vector database for content storage
        self.chroma_client = chromadb.Client()
        self.collection = self.chroma_client.create_collection(
            name="course_content",
            metadata={"hnsw:space": "cosine"}
        )

        # System state
        self.professor_profile = ProfessorProfile()
        self.course_contents: List[CourseContent] = []
        self.question_bank: List[GeneratedQuestion] = []

        # API configuration
        self.use_perplexity = use_perplexity_api
        self.api_key = api_key

        # Pedagogy assessment questions
        self.pedagogy_questions = self._load_pedagogy_questions()

        print("✅ System initialized successfully!")

    def _load_pedagogy_questions(self) -> List[Dict[str, Any]]:
        """Load predefined questions for pedagogy assessment"""
        return [
            {
                "question": "Based on the course materials, is the professor more practically oriented or theoretically focused?",
                "type": "multiple_choice",
                "options": ["Highly Practical", "Mostly Practical", "Balanced", "Mostly Theoretical", "Highly Theoretical"],
                "weight": 1.0
            },
            {
                "question": "What is the professor's experience level based on teaching methodology and content structure?",
                "type": "multiple_choice",
                "options": ["New Faculty", "Developing Faculty", "Experienced Faculty", "Veteran Faculty"],
                "weight": 0.8
            },
            {
                "question": "What assessment methodology does the professor prefer?",
                "type": "multiple_choice",
                "options": ["Frequent Small Quizzes", "Few Major Exams", "Project-Based Assessment", "Mixed Approach", "Continuous Assessment"],
                "weight": 1.0
            },
            {
                "question": "What is the overall difficulty level of the course?",
                "type": "multiple_choice",
                "options": ["Very Easy", "Easy", "Moderate", "Challenging", "Very Challenging"],
                "weight": 0.7
            },
            {
                "question": "What is the professor's interaction style in class?",
                "type": "multiple_choice",
                "options": ["Very Formal", "Formal", "Balanced", "Casual", "Very Interactive"],
                "weight": 0.6
            },
            {
                "question": "Are there any specific topics or areas the professor emphasizes as particularly important?",
                "type": "open_ended",
                "weight": 1.0
            },
            {
                "question": "What teaching methodologies does the professor employ? (e.g., case studies, practical examples, theoretical frameworks)",
                "type": "open_ended",
                "weight": 0.9
            },
            {
                "question": "Are there any special instructions or preferences mentioned by the professor for studying or evaluation?",
                "type": "open_ended",
                "weight": 0.8
            }
        ]

# 📄 DOCUMENT PROCESSING MODULE


In [ ]:

class DocumentProcessor:
    """Advanced document processing for multiple file formats"""

    def __init__(self):
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=200,
            length_function=len
        )

    def process_pdf(self, file_path: str) -> str:
        """Extract text from PDF files"""
        try:
            text = ""
            with open(file_path, 'rb') as file:
                pdf_reader = PyPDF2.PdfReader(file)
                for page in pdf_reader.pages:
                    text += page.extract_text() + "\n"
            return text.strip()
        except Exception as e:
            print(f"❌ Error processing PDF: {e}")
            return ""

    def process_docx(self, file_path: str) -> str:
        """Extract text from Word documents"""
        try:
            doc = docx.Document(file_path)
            text = "\n".join([paragraph.text for paragraph in doc.paragraphs])
            return text.strip()
        except Exception as e:
            print(f"❌ Error processing DOCX: {e}")
            return ""

    def process_text_file(self, file_path: str) -> str:
        """Process plain text files"""
        try:
            with open(file_path, 'r', encoding='utf-8') as file:
                return file.read().strip()
        except Exception as e:
            print(f"❌ Error processing text file: {e}")
            return ""

    def extract_topics(self, content: str) -> List[str]:
        """Extract key topics from content using NLP"""
        # Simple topic extraction using keyword patterns
        patterns = [
            r'Chapter \d+:?\s*(.+?)(?:\n|$)',
            r'Topic \d+:?\s*(.+?)(?:\n|$)',
            r'Lesson \d+:?\s*(.+?)(?:\n|$)',
            r'Section \d+\.?\d*:?\s*(.+?)(?:\n|$)',
            r'^\d+\.\s*(.+?)(?:\n|$)',
            r'^[A-Z][A-Za-z\s]+(?=\n|$)'
        ]

        topics = []
        for pattern in patterns:
            matches = re.findall(pattern, content, re.MULTILINE | re.IGNORECASE)
            topics.extend([match.strip() for match in matches if len(match.strip()) > 3])

        # Remove duplicates and filter
        unique_topics = list(set(topics))
        return [topic for topic in unique_topics if 5 <= len(topic) <= 100][:20]



# 🎯 PEDAGOGY MODELING ENGINE



In [ ]:
class PedagogyModeler:
    """Intelligent pedagogy analysis and modeling system"""

    def __init__(self, assistant: AgenticEducationalAssistant):
        self.assistant = assistant
        self.analysis_patterns = self._load_analysis_patterns()

    def _load_analysis_patterns(self) -> Dict[str, List[str]]:
        """Load patterns for pedagogy analysis"""
        return {
            "practical_indicators": [
                "case study", "real-world", "practical", "application", "example",
                "hands-on", "exercise", "lab", "project", "implementation"
            ],
            "theoretical_indicators": [
                "theory", "concept", "principle", "framework", "model",
                "definition", "abstract", "foundation", "fundamental"
            ],
            "assessment_indicators": {
                "quiz": ["quiz", "test", "weekly assessment", "pop quiz"],
                "exam": ["exam", "midterm", "final", "major test"],
                "project": ["project", "assignment", "portfolio", "presentation"],
                "continuous": ["participation", "attendance", "discussion", "ongoing"]
            },
            "difficulty_indicators": {
                "easy": ["introductory", "basic", "fundamental", "beginner"],
                "moderate": ["intermediate", "standard", "regular"],
                "hard": ["advanced", "complex", "challenging", "difficult", "rigorous"]
            }
        }

    def analyze_content_style(self, content: str) -> Dict[str, float]:
        """Analyze teaching style from content"""
        practical_score = 0
        theoretical_score = 0
        total_words = len(content.split())

        # Count practical indicators
        for indicator in self.analysis_patterns["practical_indicators"]:
            practical_score += content.lower().count(indicator)

        # Count theoretical indicators
        for indicator in self.analysis_patterns["theoretical_indicators"]:
            theoretical_score += content.lower().count(indicator)

        # Normalize scores
        practical_ratio = practical_score / total_words if total_words > 0 else 0
        theoretical_ratio = theoretical_score / total_words if total_words > 0 else 0

        return {
            "practical_score": practical_ratio,
            "theoretical_score": theoretical_ratio,
            "balance_score": abs(practical_ratio - theoretical_ratio)
        }

    def assess_difficulty_level(self, content: str) -> str:
        """Assess course difficulty from content"""
        difficulty_scores = {"easy": 0, "moderate": 0, "hard": 0}

        for level, indicators in self.analysis_patterns["difficulty_indicators"].items():
            for indicator in indicators:
                difficulty_scores[level] += content.lower().count(indicator)

        # Return highest scoring difficulty
        return max(difficulty_scores, key=difficulty_scores.get) or "moderate"

    def update_professor_profile(self, responses: Dict[str, str]) -> None:
        """Update professor profile based on responses and content analysis"""
        print("🔍 Analyzing professor pedagogy...")

        # Update based on direct responses
        style_mapping = {
            "Highly Practical": "practical",
            "Mostly Practical": "practical",
            "Balanced": "balanced",
            "Mostly Theoretical": "theoretical",
            "Highly Theoretical": "theoretical"
        }

        experience_mapping = {
            "New Faculty": "new",
            "Developing Faculty": "developing",
            "Experienced Faculty": "experienced",
            "Veteran Faculty": "veteran"
        }

        # Map responses to profile
        if "teaching_style" in responses:
            self.assistant.professor_profile.teaching_style = style_mapping.get(
                responses["teaching_style"], "balanced"
            )

        if "experience_level" in responses:
            self.assistant.professor_profile.experience_level = experience_mapping.get(
                responses["experience_level"], "experienced"
            )

        # Process other responses
        self.assistant.professor_profile.assessment_style = responses.get("assessment_style", "mixed")
        self.assistant.professor_profile.course_difficulty = responses.get("difficulty_level", "moderate")
        self.assistant.professor_profile.interaction_style = responses.get("interaction_style", "balanced")

        # Process open-ended responses
        if "emphasis_areas" in responses and responses["emphasis_areas"]:
            emphasis_list = [area.strip() for area in responses["emphasis_areas"].split(",")]
            self.assistant.professor_profile.emphasis_areas = emphasis_list

        if "teaching_methodology" in responses:
            self.assistant.professor_profile.teaching_methodology = responses["teaching_methodology"]

        if "special_instructions" in responses and responses["special_instructions"]:
            instructions = [inst.strip() for inst in responses["special_instructions"].split(".")]
            self.assistant.professor_profile.special_instructions = instructions

        print("✅ Professor profile updated successfully!")


# ❓ INTELLIGENT QUESTION GENERATION ENGINE


In [ ]:
class QuestionGenerator:
    """Advanced question generation using multiple AI techniques"""

    def __init__(self, assistant: AgenticEducationalAssistant):
        self.assistant = assistant
        self.question_templates = self._load_question_templates()

    def _load_question_templates(self) -> Dict[str, List[str]]:
        """Load question generation templates"""
        return {
            "mcq": [
                "What is the primary purpose of {concept}?",
                "Which of the following best describes {concept}?",
                "In the context of {topic}, what does {concept} refer to?",
                "What is the main advantage of {concept}?",
                "Which statement about {concept} is most accurate?"
            ],
            "short_answer": [
                "Explain the concept of {concept} in your own words.",
                "What are the key features of {concept}?",
                "How does {concept} relate to {related_topic}?",
                "Describe the process of {concept}.",
                "What is the significance of {concept} in {context}?"
            ],
            "essay": [
                "Analyze the impact of {concept} on {domain}.",
                "Compare and contrast {concept1} and {concept2}.",
                "Evaluate the effectiveness of {concept} in solving {problem}.",
                "Discuss the future implications of {concept}.",
                "Critically examine the role of {concept} in {field}."
            ],
            "problem_solving": [
                "Given the scenario: {scenario}, how would you apply {concept}?",
                "Design a solution using {concept} for {problem}.",
                "Calculate the {metric} when {conditions}.",
                "Solve for {variable} in the following equation: {equation}",
                "Optimize {objective} subject to {constraints}."
            ]
        }

    def extract_key_concepts(self, content: str, num_concepts: int = 10) -> List[str]:
        """Extract key concepts from content for question generation"""
        # Simple concept extraction using noun phrases and important terms
        import re

        # Find potential concepts (capitalized terms, technical terms, etc.)
        patterns = [
            r'\b[A-Z][a-z]+(?:\s+[A-Z][a-z]+)*\b',  # Proper nouns
            r'\b[a-z]+(?:ing|tion|ity|ness|ment)\b',  # Technical suffixes
            r'\b(?:the|a|an)\s+([a-z]+(?:\s+[a-z]+)*)\s+(?:is|are|was|were)\b'  # Definitions
        ]

        concepts = []
        for pattern in patterns:
            matches = re.findall(pattern, content, re.IGNORECASE)
            concepts.extend([match.strip() for match in matches if isinstance(match, str)])

        # Filter and clean
        filtered_concepts = []
        for concept in concepts:
            if 3 <= len(concept) <= 50 and concept.lower() not in ['the', 'and', 'for', 'are', 'was']:
                filtered_concepts.append(concept)

        # Remove duplicates and return top concepts
        unique_concepts = list(set(filtered_concepts))
        return unique_concepts[:num_concepts]

    def generate_mcq_question(self, concept: str, content: str) -> GeneratedQuestion:
        """Generate multiple choice question for a concept"""
        # Select template based on professor style
        template = self.question_templates["mcq"][0]  # Simple selection

        question_text = template.format(concept=concept)

        # Generate options (simplified for demo)
        options = [
            f"A primary function related to {concept}",
            f"A secondary aspect of {concept}",
            f"An alternative to {concept}",
            f"The opposite of {concept}"
        ]

        return GeneratedQuestion(
            question=question_text,
            question_type="mcq",
            difficulty="medium",
            topic=concept,
            options=options,
            answer=options[0],  # First option as correct answer
            explanation=f"This question tests understanding of {concept}.",
            estimated_time=3
        )

    def generate_questions_for_content(self, content: CourseContent, num_questions: int = 5) -> List[GeneratedQuestion]:
        """Generate multiple questions for given content"""
        print(f"🎯 Generating {num_questions} questions for: {content.title}")

        questions = []
        concepts = self.extract_key_concepts(content.content, num_questions * 2)

        # Generate different types of questions
        question_types = ["mcq", "short_answer", "essay"] if len(concepts) > 3 else ["mcq", "short_answer"]

        for i in range(min(num_questions, len(concepts))):
            concept = concepts[i]
            question_type = question_types[i % len(question_types)]

            if question_type == "mcq":
                question = self.generate_mcq_question(concept, content.content)
            else:
                # Generate other question types (simplified)
                template = self.question_templates[question_type][0]
                question_text = template.format(concept=concept)

                question = GeneratedQuestion(
                    question=question_text,
                    question_type=question_type,
                    difficulty="medium",
                    topic=concept,
                    answer=f"Answer should explain {concept} in detail.",
                    explanation=f"This {question_type} question assesses deep understanding of {concept}."
                )

            questions.append(question)

        print(f"✅ Generated {len(questions)} questions successfully!")
        return questions


# 🎓 STUDY GUIDE GENERATOR

In [ ]:
class StudyGuideGenerator:
    """Generate comprehensive study materials and guides"""

    def __init__(self, assistant: AgenticEducationalAssistant):
        self.assistant = assistant

    def generate_topic_priorities(self) -> List[Tuple[str, str, float]]:
        """Generate prioritized list of topics based on professor emphasis"""
        priorities = []

        # Extract all topics from course content
        all_topics = []
        for content in self.assistant.course_contents:
            all_topics.extend(content.topics)

        # Score topics based on professor emphasis
        emphasis_areas = self.assistant.professor_profile.emphasis_areas

        for topic in set(all_topics):
            score = 1.0  # Base score

            # Boost score if mentioned in emphasis areas
            for emphasis in emphasis_areas:
                if emphasis.lower() in topic.lower() or topic.lower() in emphasis.lower():
                    score += 2.0

            # Adjust based on professor style
            if self.assistant.professor_profile.teaching_style == "practical":
                if any(word in topic.lower() for word in ["application", "example", "case", "practical"]):
                    score += 1.0
            elif self.assistant.professor_profile.teaching_style == "theoretical":
                if any(word in topic.lower() for word in ["theory", "concept", "principle", "model"]):
                    score += 1.0

            # Categorize importance
            if score >= 3.0:
                importance = "High"
            elif score >= 2.0:
                importance = "Medium"
            else:
                importance = "Low"

            priorities.append((topic, importance, score))

        # Sort by score
        priorities.sort(key=lambda x: x[2], reverse=True)
        return priorities[:15]  # Return top 15 topics

    def generate_study_schedule(self, days_until_exam: int = 14) -> Dict[str, List[str]]:
        """Generate a study schedule based on topic priorities"""
        priorities = self.generate_topic_priorities()

        # Distribute topics across available days
        schedule = {}
        topics_per_day = max(1, len(priorities) // days_until_exam)

        current_day = 1
        for i, (topic, importance, score) in enumerate(priorities):
            day_key = f"Day {current_day}"
            if day_key not in schedule:
                schedule[day_key] = []

            schedule[day_key].append(f"{topic} ({importance} Priority)")

            if len(schedule[day_key]) >= topics_per_day:
                current_day += 1
                if current_day > days_until_exam:
                    break

        return schedule

    def generate_quick_study_guide(self) -> str:
        """Generate a comprehensive quick study guide"""
        guide = "# 📚 Quick Study Guide\n\n"

        # Professor-specific study tips
        guide += "## 🎯 Professor-Specific Study Strategy\n\n"

        profile = self.assistant.professor_profile

        if profile.teaching_style == "practical":
            guide += "- **Focus on practical applications and real-world examples**\n"
            guide += "- **Practice case studies and problem-solving scenarios**\n"
        elif profile.teaching_style == "theoretical":
            guide += "- **Master fundamental concepts and theoretical frameworks**\n"
            guide += "- **Understand underlying principles and relationships**\n"
        else:
            guide += "- **Balance theoretical understanding with practical applications**\n"

        if profile.assessment_style in ["frequent_quizzes", "quiz"]:
            guide += "- **Review regularly and practice short-form questions**\n"
        elif profile.assessment_style in ["major_exams", "exam"]:
            guide += "- **Focus on comprehensive review and long-form preparation**\n"

        # Topic priorities
        guide += "\n## 🎯 Priority Topics\n\n"
        priorities = self.generate_topic_priorities()

        for importance in ["High", "Medium", "Low"]:
            relevant_topics = [topic for topic, imp, score in priorities if imp == importance]
            if relevant_topics:
                guide += f"### {importance} Priority:\n"
                for topic in relevant_topics[:5]:  # Limit to 5 per category
                    guide += f"- {topic}\n"
                guide += "\n"

        # Special instructions
        if profile.special_instructions:
            guide += "## ⚠️ Special Instructions from Professor\n\n"
            for instruction in profile.special_instructions:
                guide += f"- {instruction}\n"
            guide += "\n"

        # Study schedule
        guide += "## 📅 Recommended Study Schedule\n\n"
        schedule = self.generate_study_schedule()

        for day, topics in list(schedule.items())[:7]:  # Show first week
            guide += f"**{day}:**\n"
            for topic in topics:
                guide += f"  - {topic}\n"
            guide += "\n"

        return guide


# 💬 INTERACTIVE CHATBOT INTERFACE

In [ ]:
class InteractiveChatbot:
    """Main chatbot interface for student interaction"""

    def __init__(self, assistant: AgenticEducationalAssistant):
        self.assistant = assistant
        self.doc_processor = DocumentProcessor()
        self.pedagogy_modeler = PedagogyModeler(assistant)
        self.question_generator = QuestionGenerator(assistant)
        self.study_guide_generator = StudyGuideGenerator(assistant)

        self.conversation_state = "initial"
        self.pedagogy_responses = {}
        self.current_question_index = 0

    def upload_document(self, file_path: str, content_type: str = "slides") -> bool:
        """Upload and process a document"""
        print(f"📄 Processing document: {file_path}")

        # Process based on file extension
        if file_path.endswith('.pdf'):
            content = self.doc_processor.process_pdf(file_path)
        elif file_path.endswith('.docx'):
            content = self.doc_processor.process_docx(file_path)
        elif file_path.endswith('.txt'):
            content = self.doc_processor.process_text_file(file_path)
        else:
            print("❌ Unsupported file format")
            return False

        if not content:
            print("❌ Failed to extract content")
            return False

        # Extract topics and create course content
        topics = self.doc_processor.extract_topics(content)

        course_content = CourseContent(
            title=Path(file_path).stem,
            content=content,
            content_type=content_type,
            topics=topics
        )

        self.assistant.course_contents.append(course_content)

        # Add to vector database
        chunks = self.doc_processor.text_splitter.split_text(content)
        embeddings = self.assistant.embedding_model.encode(chunks)

        for i, (chunk, embedding) in enumerate(zip(chunks, embeddings)):
            self.assistant.collection.add(
                embeddings=[embedding.tolist()],
                documents=[chunk],
                ids=[f"{course_content.title}_{i}"],
                metadatas=[{"title": course_content.title, "type": content_type}]
            )

        print(f"✅ Successfully processed document with {len(topics)} topics")
        return True

    def start_pedagogy_assessment(self) -> str:
        """Begin the pedagogy assessment process"""
        self.conversation_state = "pedagogy_assessment"
        self.current_question_index = 0

        response = "🎓 **Welcome to your Agentic AI Educational Assistant!**\n\n"
        response += "I've analyzed your course materials and now I need to understand your professor's teaching style better.\n\n"
        response += "This will help me create personalized study materials and practice questions that match your professor's approach.\n\n"
        response += "**Let's start with the first question:**\n\n"
        response += self._get_current_pedagogy_question()

        return response

    def _get_current_pedagogy_question(self) -> str:
        """Get the current pedagogy assessment question"""
        if self.current_question_index >= len(self.assistant.pedagogy_questions):
            return self._complete_pedagogy_assessment()

        question_data = self.assistant.pedagogy_questions[self.current_question_index]
        question_text = f"**Question {self.current_question_index + 1}:** {question_data['question']}\n\n"

        if question_data['type'] == 'multiple_choice':
            question_text += "**Options:**\n"
            for i, option in enumerate(question_data['options'], 1):
                question_text += f"{i}. {option}\n"
            question_text += "\nPlease respond with the number of your choice."
        else:
            question_text += "Please provide your detailed response."

        return question_text

    def process_pedagogy_response(self, response: str) -> str:
        """Process response to pedagogy assessment"""
        question_data = self.assistant.pedagogy_questions[self.current_question_index]

        # Store response
        if question_data['type'] == 'multiple_choice':
            try:
                choice_num = int(response.strip()) - 1
                if 0 <= choice_num < len(question_data['options']):
                    selected_option = question_data['options'][choice_num]
                    self.pedagogy_responses[self._get_response_key()] = selected_option
                else:
                    return "❌ Invalid choice. Please select a valid option number."
            except ValueError:
                return "❌ Please respond with a number corresponding to your choice."
        else:
            self.pedagogy_responses[self._get_response_key()] = response.strip()

        # Move to next question
        self.current_question_index += 1

        if self.current_question_index >= len(self.assistant.pedagogy_questions):
            return self._complete_pedagogy_assessment()
        else:
            return f"✅ Response recorded!\n\n{self._get_current_pedagogy_question()}"

    def _get_response_key(self) -> str:
        """Get the key for storing pedagogy response"""
        question_mappings = {
            0: "teaching_style",
            1: "experience_level",
            2: "assessment_style",
            3: "difficulty_level",
            4: "interaction_style",
            5: "emphasis_areas",
            6: "teaching_methodology",
            7: "special_instructions"
        }
        return question_mappings.get(self.current_question_index, f"question_{self.current_question_index}")

    def _complete_pedagogy_assessment(self) -> str:
        """Complete the pedagogy assessment and build professor profile"""
        # Update professor profile
        self.pedagogy_modeler.update_professor_profile(self.pedagogy_responses)

        # Change conversation state
        self.conversation_state = "active"

        response = "🎉 **Pedagogy Assessment Complete!**\n\n"
        response += "I've successfully created a model of your professor's teaching style. Here's what I learned:\n\n"

        profile = self.assistant.professor_profile
        response += f"**Teaching Style:** {profile.teaching_style.title()}\n"
        response += f"**Experience Level:** {profile.experience_level.title()}\n"
        response += f"**Assessment Style:** {profile.assessment_style.title().replace('_', ' ')}\n"
        response += f"**Course Difficulty:** {profile.course_difficulty.title()}\n\n"

        if profile.emphasis_areas:
            response += f"**Emphasis Areas:** {', '.join(profile.emphasis_areas)}\n\n"

        response += "**🚀 I'm now ready to help you with:**\n"
        response += "- Generate practice questions\n"
        response += "- Create study guides\n"
        response += "- Provide topic priorities\n"
        response += "- Help with assignments\n"
        response += "- Prepare exam materials\n\n"
        response += "What would you like me to help you with?"

        return response

    def handle_student_query(self, query: str) -> str:
        """Handle general student queries and requests"""
        query_lower = query.lower()

        # Intent detection and response routing
        if any(word in query_lower for word in ["question", "practice", "quiz", "test"]):
            return self._handle_question_request(query)
        elif any(word in query_lower for word in ["study guide", "guide", "summary"]):
            return self._generate_study_guide_response()
        elif any(word in query_lower for word in ["priority", "important", "focus"]):
            return self._generate_priority_topics_response()
        elif any(word in query_lower for word in ["schedule", "plan", "timeline"]):
            return self._generate_study_schedule_response()
        elif any(word in query_lower for word in ["exam", "preparation", "review"]):
            return self._handle_exam_preparation(query)
        else:
            return self._handle_general_query(query)

    def _handle_question_request(self, query: str) -> str:
        """Handle requests for practice questions"""
        # Determine number of questions
        num_questions = 5  # Default
        if "one" in query.lower() or "1" in query:
            num_questions = 1
        elif "few" in query.lower() or any(str(i) in query for i in range(2, 6)):
            num_questions = 3
        elif "many" in query.lower() or "10" in query:
            num_questions = 10

        if not self.assistant.course_contents:
            return "❌ No course materials uploaded yet. Please upload your course materials first."

        # Generate questions from all content
        all_questions = []
        for content in self.assistant.course_contents:
            questions = self.question_generator.generate_questions_for_content(content, num_questions // len(self.assistant.course_contents) + 1)
            all_questions.extend(questions)

        # Select and format questions
        selected_questions = all_questions[:num_questions]
        self.assistant.question_bank.extend(selected_questions)

        response = f"📝 **Generated {len(selected_questions)} Practice Questions:**\n\n"

        for i, question in enumerate(selected_questions, 1):
            response += f"**Question {i}** ({question.difficulty} - {question.question_type.upper()}):\n"
            response += f"{question.question}\n\n"

            if question.options:
                for j, option in enumerate(question.options, 1):
                    response += f"  {j}. {option}\n"
                response += "\n"

            response += f"*Topic: {question.topic}*\n"
            response += f"*Estimated Time: {question.estimated_time} minutes*\n\n"
            response += "---\n\n"

        response += "💡 **Pro Tip:** Based on your professor's style, focus on "
        if self.assistant.professor_profile.teaching_style == "practical":
            response += "real-world applications and practical examples!"
        elif self.assistant.professor_profile.teaching_style == "theoretical":
            response += "understanding core concepts and theoretical foundations!"
        else:
            response += "balancing theory with practical applications!"

        return response

    def _generate_study_guide_response(self) -> str:
        """Generate comprehensive study guide"""
        guide = self.study_guide_generator.generate_quick_study_guide()
        return guide

    def _generate_priority_topics_response(self) -> str:
        """Generate prioritized topics list"""
        priorities = self.study_guide_generator.generate_topic_priorities()

        response = "🎯 **Topic Priorities Based on Professor's Emphasis:**\n\n"

        current_importance = None
        for topic, importance, score in priorities:
            if importance != current_importance:
                response += f"## {importance} Priority Topics\n\n"
                current_importance = importance

            response += f"- **{topic}** (Score: {score:.1f})\n"

        response += "\n💡 **Study Recommendation:** Start with High Priority topics and allocate more time to areas your professor specifically emphasized."

        return response

    def _generate_study_schedule_response(self) -> str:
        """Generate study schedule"""
        schedule = self.study_guide_generator.generate_study_schedule()

        response = "📅 **Personalized Study Schedule:**\n\n"

        for day, topics in schedule.items():
            response += f"### {day}\n"
            for topic in topics:
                response += f"- {topic}\n"
            response += "\n"

        response += "⏰ **Time Management Tips:**\n"
        response += f"- Allocate {2 if self.assistant.professor_profile.course_difficulty == 'challenging' else 1.5} hours per day\n"
        response += "- Take 10-minute breaks every hour\n"
        response += "- Review previous day's topics for 15 minutes\n"

        return response

    def _handle_exam_preparation(self, query: str) -> str:
        """Handle exam preparation requests"""
        response = "📋 **Comprehensive Exam Preparation Plan:**\n\n"

        # Study guide
        response += "## 📚 Study Guide\n"
        guide = self.study_guide_generator.generate_quick_study_guide()
        response += guide + "\n\n"

        # Sample questions
        response += "## 📝 Sample Questions\n"
        if self.assistant.course_contents:
            questions = []
            for content in self.assistant.course_contents[:2]:  # Limit to 2 contents
                content_questions = self.question_generator.generate_questions_for_content(content, 3)
                questions.extend(content_questions)

            for i, question in enumerate(questions[:5], 1):
                response += f"**{i}.** {question.question}\n\n"

        # Final tips
        response += "## 🎯 Final Exam Tips\n"
        response += f"- **Professor Style:** {self.assistant.professor_profile.teaching_style} - adjust your preparation accordingly\n"
        response += f"- **Assessment Style:** {self.assistant.professor_profile.assessment_style} - practice this format\n"
        response += "- **Time Management:** Practice under timed conditions\n"
        response += "- **Review:** Go through professor's emphasis areas one more time\n"

        return response

    def _handle_general_query(self, query: str) -> str:
        """Handle general queries using retrieval"""
        # Search course content for relevant information
        query_embedding = self.assistant.embedding_model.encode([query])

        try:
            results = self.assistant.collection.query(
                query_embeddings=query_embedding.tolist(),
                n_results=3
            )

            if results['documents'] and results['documents'][0]:
                context = "\n".join(results['documents'][0])

                response = "📖 **Based on your course materials:**\n\n"
                response += f"**Query:** {query}\n\n"
                response += f"**Relevant Information:**\n{context[:500]}...\n\n"
                response += "💡 **Need more specific help?** Try asking for:\n"
                response += "- Practice questions on this topic\n"
                response += "- Study guide for this area\n"
                response += "- Priority level of this topic\n"

                return response
            else:
                return "❓ I couldn't find specific information about that in your course materials. Could you please be more specific or ask for practice questions, study guides, or topic priorities?"

        except Exception as e:
            return f"❓ I'm having trouble searching the course materials. Please try rephrasing your question or ask for practice questions, study guides, or topic priorities. Error: {str(e)}"


# 🚀 MAIN EXECUTION AND DEMO

In [ ]:
def create_sample_content():
    """Create sample course content for demonstration"""

    # Create sample files
    sample_content = {
        "Data_Structures_Lecture_1.txt": """
        Data Structures - Introduction

        Chapter 1: Introduction to Data Structures

        What are Data Structures?
        Data structures are ways of organizing and storing data in a computer so that it can be accessed and used efficiently. They define the relationship between data elements and the operations that can be performed on them.

        Types of Data Structures:
        1. Linear Data Structures
           - Arrays
           - Linked Lists
           - Stacks
           - Queues

        2. Non-Linear Data Structures
           - Trees
           - Graphs
           - Hash Tables

        Key Concepts:
        - Time Complexity: How execution time grows with input size
        - Space Complexity: How memory usage grows with input size
        - Big O Notation: Mathematical notation for algorithm efficiency

        Arrays:
        Arrays are a collection of elements stored at contiguous memory locations. The idea is to store multiple items of the same type together.

        Advantages of Arrays:
        - Random access to elements
        - Cache friendly
        - Easy to implement

        Disadvantages of Arrays:
        - Fixed size
        - Insertion and deletion are expensive

        Real-world Applications:
        Arrays are used in implementing other data structures like stacks, queues, and hash tables. They are fundamental in computer graphics, numerical computing, and database systems.
        """,

        "Course_Policy.txt": """
        Course Policy - CS 201: Data Structures and Algorithms

        Instructor: Dr. Sarah Johnson
        Office Hours: Monday & Wednesday 2-4 PM

        Course Objectives:
        This course provides a comprehensive introduction to data structures and algorithms. Students will learn to analyze, design, and implement efficient algorithms and data structures.

        Assessment Methods:
        - Weekly Quizzes: 20%
        - Mid-term Exam: 25%
        - Final Exam: 35%
        - Programming Assignments: 20%

        Important Notes:
        - Emphasis on practical implementation and problem-solving
        - Students must demonstrate proficiency in both theoretical concepts and coding
        - Late submissions will be penalized 10% per day
        - Collaboration is encouraged for learning but individual work required for assessments

        Required Topics:
        - Array operations and analysis
        - Linked list implementations
        - Stack and queue applications
        - Tree traversal algorithms
        - Graph algorithms
        - Sorting and searching algorithms
        - Hash table design

        Programming Language: Python (primary), C++ (optional)

        Study Recommendations:
        - Practice coding problems daily
        - Focus on understanding time and space complexity
        - Work through real-world examples
        - Join study groups for collaborative learning
        """
    }

    # Write sample files
    for filename, content in sample_content.items():
        with open(filename, 'w', encoding='utf-8') as f:
            f.write(content)

    print("📝 Sample course content created!")
    return list(sample_content.keys())

def run_interactive_demo():
    """Run an interactive demonstration of the system"""

    print("🎓 AGENTIC AI EDUCATIONAL ASSISTANT DEMO")
    print("=" * 50)

    # Initialize system
    assistant = AgenticEducationalAssistant()
    chatbot = InteractiveChatbot(assistant)

    # Create sample content
    sample_files = create_sample_content()

    # Upload sample documents
    print("\n📤 Uploading sample course materials...")
    for file in sample_files:
        success = chatbot.upload_document(file, "slides" if "Lecture" in file else "policy")
        if success:
            print(f"✅ {file} uploaded successfully")

    # Start pedagogy assessment
    print("\n🔍 Starting Pedagogy Assessment...")
    print("-" * 30)

    assessment_response = chatbot.start_pedagogy_assessment()
    print(assessment_response)

    # Simulate assessment responses
    print("\n🤖 Simulating assessment responses...")
    sample_responses = [
        "2",  # Mostly Practical
        "3",  # Experienced Faculty
        "1",  # Frequent Small Quizzes
        "3",  # Moderate difficulty
        "4",  # Casual interaction
        "algorithms, problem-solving, coding practice",  # Emphasis areas
        "Uses real-world examples, encourages hands-on coding, provides immediate feedback",  # Teaching methodology
        "Focus on practical implementation. Practice coding daily. Understand time complexity."  # Special instructions
    ]

    for i, response in enumerate(sample_responses):
        print(f"\n📝 Response {i+1}: {response}")
        result = chatbot.process_pedagogy_response(response)
        print(result)
        if "complete" in result.lower():
            break

    # Demonstrate various features
    print("\n" + "="*50)
    print("🚀 FEATURE DEMONSTRATIONS")
    print("="*50)

    # 1. Generate practice questions
    print("\n1️⃣ GENERATING PRACTICE QUESTIONS")
    print("-" * 30)
    question_response = chatbot.handle_student_query("Generate 3 practice questions")
    print(question_response)

    # 2. Create study guide
    print("\n2️⃣ CREATING STUDY GUIDE")
    print("-" * 30)
    study_guide_response = chatbot.handle_student_query("Create a study guide")
    print(study_guide_response)

    # 3. Show topic priorities
    print("\n3️⃣ TOPIC PRIORITIES")
    print("-" * 30)
    priority_response = chatbot.handle_student_query("What topics should I prioritize?")
    print(priority_response)

    # 4. Generate study schedule
    print("\n4️⃣ STUDY SCHEDULE")
    print("-" * 30)
    schedule_response = chatbot.handle_student_query("Create a study schedule")
    print(schedule_response)

    # 5. Exam preparation
    print("\n5️⃣ EXAM PREPARATION")
    print("-" * 30)
    exam_response = chatbot.handle_student_query("Help me prepare for the exam")
    print(exam_response[:1000] + "..." if len(exam_response) > 1000 else exam_response)

    print("\n" + "="*50)
    print("✅ DEMO COMPLETED SUCCESSFULLY!")
    print("="*50)

    return assistant, chatbot


# 📊 SYSTEM ANALYTICS AND MONITORING

In [ ]:
class SystemAnalytics:
    """Monitor and analyze system performance"""

    def __init__(self, assistant: AgenticEducationalAssistant):
        self.assistant = assistant
        self.metrics = {
            "questions_generated": 0,
            "documents_processed": 0,
            "student_interactions": 0,
            "study_guides_created": 0
        }
        self.start_time = datetime.now()

    def log_interaction(self, interaction_type: str):
        """Log user interactions"""
        if interaction_type in self.metrics:
            self.metrics[interaction_type] += 1

    def generate_analytics_report(self) -> str:
        """Generate comprehensive analytics report"""
        runtime = datetime.now() - self.start_time

        report = "📊 **System Analytics Report**\n\n"
        report += f"**Runtime:** {runtime}\n"
        report += f"**Documents Processed:** {len(self.assistant.course_contents)}\n"
        report += f"**Questions Generated:** {len(self.assistant.question_bank)}\n"
        report += f"**Vector Database Size:** {self.assistant.collection.count()} entries\n\n"

        # Professor profile summary
        profile = self.assistant.professor_profile
        report += "**Professor Profile:**\n"
        report += f"- Teaching Style: {profile.teaching_style}\n"
        report += f"- Experience: {profile.experience_level}\n"
        report += f"- Assessment Style: {profile.assessment_style}\n"
        report += f"- Course Difficulty: {profile.course_difficulty}\n\n"

        # Content analysis
        total_topics = sum(len(content.topics) for content in self.assistant.course_contents)
        report += f"**Content Analysis:**\n"
        report += f"- Total Topics Identified: {total_topics}\n"
        report += f"- Average Topics per Document: {total_topics / len(self.assistant.course_contents) if self.assistant.course_contents else 0:.1f}\n"

        return report


# 🎯 EXECUTION ENTRY POINT

In [ ]:
if __name__ == "__main__":
    print("🚀 Starting Agentic AI Educational Assistant...")

    # Run the interactive demo
    assistant, chatbot = run_interactive_demo()

    # Generate analytics
    analytics = SystemAnalytics(assistant)
    analytics_report = analytics.generate_analytics_report()

    print("\n📊 SYSTEM ANALYTICS")
    print("-" * 20)
    print(analytics_report)

    # Save system state
    print("\n💾 Saving system state...")

    # Save professor profile
    with open('professor_profile.json', 'w') as f:
        json.dump(asdict(assistant.professor_profile), f, indent=2)

    # Save question bank
    with open('question_bank.json', 'w') as f:
        json.dump([asdict(q) for q in assistant.question_bank], f, indent=2)

    print("✅ System state saved successfully!")

    # Interactive mode (optional)
    print("\n💬 INTERACTIVE MODE")
    print("-" * 20)
    print("You can now interact with the chatbot directly!")
    print("Type 'exit' to quit, 'help' for available commands\n")

    while True:
        try:
            user_input = input("🎓 Student: ").strip()

            if user_input.lower() == 'exit':
                print("👋 Goodbye! Happy studying!")
                break
            elif user_input.lower() == 'help':
                print("""
📚 Available Commands:
- "Generate practice questions" - Create custom practice questions
- "Create study guide" - Generate comprehensive study guide
- "Show topic priorities" - Display important topics
- "Create study schedule" - Generate personalized study plan
- "Help with exam preparation" - Comprehensive exam prep
- "Upload [filename]" - Upload new course materials
- "Analytics" - Show system analytics
- "exit" - Quit the program
                """)
                continue
            elif user_input.lower() == 'analytics':
                print(analytics.generate_analytics_report())
                continue
            elif user_input.lower().startswith('upload'):
                filename = user_input.split(' ', 1)[1] if ' ' in user_input else ''
                if filename and os.path.exists(filename):
                    success = chatbot.upload_document(filename)
                    print("✅ Document uploaded!" if success else "❌ Upload failed!")
                else:
                    print("❌ File not found or filename not provided")
                continue

            # Process regular queries
            response = chatbot.handle_student_query(user_input)
            print(f"\n🤖 AI Assistant: {response}\n")

            analytics.log_interaction("student_interactions")

        except KeyboardInterrupt:
            print("\n\n👋 Goodbye! Happy studying!")
            break
        except Exception as e:
            print(f"❌ Error: {e}")
            continue


🚀 Starting Agentic AI Educational Assistant...
🎓 AGENTIC AI EDUCATIONAL ASSISTANT DEMO
🔧 Initializing Agentic Educational Assistant...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

Device set to use cpu


✅ System initialized successfully!
📝 Sample course content created!

📤 Uploading sample course materials...
📄 Processing document: Data_Structures_Lecture_1.txt
✅ Successfully processed document with 1 topics
✅ Data_Structures_Lecture_1.txt uploaded successfully
📄 Processing document: Course_Policy.txt
✅ Successfully processed document with 0 topics
✅ Course_Policy.txt uploaded successfully

🔍 Starting Pedagogy Assessment...
------------------------------
🎓 **Welcome to your Agentic AI Educational Assistant!**

I've analyzed your course materials and now I need to understand your professor's teaching style better.

This will help me create personalized study materials and practice questions that match your professor's approach.

**Let's start with the first question:**

**Question 1:** Based on the course materials, is the professor more practically oriented or theoretically focused?

**Options:**
1. Highly Practical
2. Mostly Practical
3. Balanced
4. Mostly Theoretical
5. Highly Theor

KeyError: 'domain'